In [1]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
import asyncio
import os

import requests
import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from typing import Literal, Union
from pydantic.config import ConfigDict

from src.models import *

from src.utilities import *
from src.bubble import *

import openai
import instructor

Retrieved company Darty : 1707313014508x102198350946437700
Retrieved project Expérience utilisateur de Darty : 1710335239022x546888753774592000


In [2]:
prompts_path = "Prompts/fr/"

# Company informations

In [3]:
with open(prompts_path+'prompt_company_informations.txt') as f:
    template = PromptTemplate.from_template(f.read())
prompt_company_informations = template.invoke(context)

class CompanyContext(BaseModel):
    context: str = Field(description="Description du contexte de l'entreprise.")

    def __str__(self):
        return self.context

res = get_analysis(prompt_company_informations, CompanyContext)
context['context'] = res.context
print(res)

Darty


# Categories creation

In [4]:
context['cible'] = "clients"


In [15]:
feedbacks_df = get("Feedback", max_objects=1000)


In [6]:
companies_df = get("Company", constraints = [])

In [7]:
context = {**context, **companies_df[companies_df["Name"]==context["company"]].iloc[0].to_dict()}
context

{'company': 'Darty',
 'project': 'Expérience utilisateur de Darty',
 'site': 'www.darty.com',
 'source': 'Trustpilot',
 'context': 'Darty',
 'cible': 'clients',
 'Context': 'Fondée en 1957, Darty est une enseigne française spécialisée dans la distribution d\'électroménager, d\'équipements électroniques et de produits culturels. Rachetée par la Fnac en 2016, elle est aujourd\'hui l\'un des leaders européens de la distribution omnicanale.\n\nÉvènements récents:\n\n    2016: Rachat par la Fnac et création du groupe Fnac Darty.\n    2017: Lancement de la marketplace Darty.com.\n    2018: Déploiement du "Contrat de Confiance Fnac Darty" dans tous les magasins.\n    2019: Lancement de l\'offre de services "Darty+."\n    2020: Accélération de la transformation digitale du groupe.\n    2021: Acquisition de Mistergooddeal, spécialiste du e-commerce en produits reconditionnés.\n    2022: Lancement de la Fnac Darty Academy, une plateforme de formation en ligne.\n\nConcurrents:\n\n    Boulanger\n 

In [14]:
feedbacks_df

,Company,Content,Date,Project,Location,Sentiment,Source,Aspects,_id
0,1707313014508x102198350946437700,Achat Magic Keyboard avec Touch ID et pave num...,2024-02-07 00:00:00+00:00,1710335239022x546888753774592000,FR,5,Trustpilot,[],1710335410193x146397114086602720
1,1707313014508x102198350946437700,Très très apprécié d’installer la…\nTrès très ...,2024-01-31 00:00:00+00:00,1710335239022x546888753774592000,FR,5,Trustpilot,NaN,1710335410195x175970366374120320
2,1707313014508x102198350946437700,Machine à laver Bosch\nCommandée sur le site d...,2024-01-05 00:00:00+00:00,1710335239022x546888753774592000,FR,5,Trustpilot,NaN,1710335410202x536776314861663740
3,1707313014508x102198350946437700,VENTE FORCEE ?\nLe 8 nov 2023 j'achète un ordi...,2023-11-08 00:00:00+00:00,1710335239022x546888753774592000,FR,1,Trustpilot,NaN,1710335410203x151859605597644900
4,1707313014508x102198350946437700,Fausse vente de crédit gratuit\nFausse vente d...,2023-10-06 00:00:00+00:00,1710335239022x546888753774592000,FR,1,Trustpilot,NaN,1710335410205x126748698162511440
...,...,...,...,...,...,...,...,...,...
95,1707313014508x102198350946437700,Service après vente Super\nService après vente...,2024-01-18 00:00:00+00:00,1710335239022x546888753774592000,FR,5,Trustpilot,NaN,1710335410869x108759416795084500
96,1707313014508x102198350946437700,Produit toujours pas reçu 2 mois après la comm...,2023-12-14 00:00:00+00:00,1710335239022x546888753774592000,FR,1,Trustpilot,NaN,1710335410872x283990491652819970
97,1707313014508x102198350946437700,Bon choix\nHotte installée facilement sous un ...,2024-01-22 00:00:00+00:00,1710335239022x546888753774592000,FR,4,Trustpilot,NaN,1710335410873x629562654646477800
98,1707313014508x102198350946437700,Je commande un réfrigérateur Liebherr…\nJe com...,2024-01-26 00:00:00+00:00,1710335239022x546888753774592000,FR,1,Trustpilot,NaN,1710335410874x858291449739420300


In [16]:
with open(prompts_path+'prompt_categories_creation.txt') as f:
    template = PromptTemplate.from_template(f.read())
prompt_categories_creation = template.invoke({'feedbacks': '\n\n'.join(feedbacks_df["Content"]), "challenges": '- '+'- '.join(context["Challenges"]),**context})

class InsightCategory(BaseModel):
    name: str = Field(description="Nom de la catégorie.")
    #description: str = Field(description="Phrase de description de la catégorie.")
    sub_categories: List[str] = Field(description="Liste des sous-catégories.")

    def __str__(self):
        return self.name + " :\n- " + '\n- '.join([str(x) for x in self.sub_categories])

class InsightCategories(BaseModel):
    categories: List[InsightCategory] = Field(description="Liste des catégories.")

    def __str__(self):
        return '\n\n'.join([str(x) for x in self.categories])

res = get_analysis(prompt_categories_creation, InsightCategories)
print(res)

Qualité du produit :
- Fonctionnalité
- Durabilité
- Conformité
- Performance
- Design
- Rapport qualité-prix

Service client :
- Amabilité
- Efficacité
- Disponibilité
- Compétence
- Réactivité
- Communication

Expérience d'achat :
- Navigation sur le site
- Conseil en magasin
- Processus de commande
- Options de livraison
- Facilité de retour
- Clarté des informations

Livraison et installation :
- Rapidité
- Fiabilité
- Professionnalisme des livreurs
- Qualité de l'installation
- Respect du créneau horaire
- Conditionnement du produit

Service après-vente :
- Gestion des réclamations
- Délai de réparation
- Qualité de la réparation
- Prise en charge de la garantie
- Solution proposée
- Suivi du dossier

Programmes et offres :
- Promotions
- Fidélité
- Garanties
- Abonnements
- Offres spéciales
- Disponibilité des produits


In [19]:
res.categories

[InsightCategory(name='Qualité du produit', sub_categories=['Fonctionnalité', 'Durabilité', 'Conformité', 'Performance', 'Design', 'Rapport qualité-prix']),
 InsightCategory(name='Service client', sub_categories=['Amabilité', 'Efficacité', 'Disponibilité', 'Compétence', 'Réactivité', 'Communication']),
 InsightCategory(name="Expérience d'achat", sub_categories=['Navigation sur le site', 'Conseil en magasin', 'Processus de commande', 'Options de livraison', 'Facilité de retour', 'Clarté des informations']),
 InsightCategory(name='Livraison et installation', sub_categories=['Rapidité', 'Fiabilité', 'Professionnalisme des livreurs', "Qualité de l'installation", 'Respect du créneau horaire', 'Conditionnement du produit']),
 InsightCategory(name='Service après-vente', sub_categories=['Gestion des réclamations', 'Délai de réparation', 'Qualité de la réparation', 'Prise en charge de la garantie', 'Solution proposée', 'Suivi du dossier']),
 InsightCategory(name='Programmes et offres', sub_cate

In [20]:
res.categories.append(InsightCategory(name="", sub_categories=[]))
for categ in res.categories:
    categ.sub_categories.append("")

In [21]:
for categorie in tqdm(res.categories):
    categ_id = bubble_client.create(
        "Category",
        {
            "Company": COMPANY_ID,
            "Name": categorie.name,
            "Project": PROJECT_ID,
        })
    
    for sub_categ in categorie.sub_categories:
        bubble_client.create(
            "SubCategory",
            {
                "Company": COMPANY_ID,
                "Category": categ_id,
                "Name": sub_categ,
                "Project": PROJECT_ID,
                #"Description": sub_categorie.description,
            })

  0%|          | 0/7 [00:00<?, ?it/s]